In [1]:
import numpy as np
import os, sys
import bids
import pandas as pd

In [2]:
######calculating fd 
abspath = os.path.abspath('__file__')
dname = os.path.dirname(abspath)
dname = os.path.dirname(dname)
os.environ["ROOTDIR"] = dname  # seth path
rootdir = os.environ["ROOTDIR"]
if hasattr(sys, "ps1"):
    options = {}
    workdir = os.environ["ROOTDIR"]
    derivat = os.path.join(workdir,"pilotStudydata")
    output  = os.path.join(workdir,"data")
    masks = os.path.join(workdir,"masks")
    participants = []
# else :
#     options = parse()
#     participants = options.participants
#     workdir = options.workdir
#     rawdata = options.rawdata
#     derivat = options.derivatives
#     output  = options.output
#     script_path = os.path.dirname(__file__)
#     masks = os.path.join(script_path,"masks")
print('Rawdata: ', derivat)
  
bidslayout = bids.BIDSLayout(derivat, derivatives=True, validate=False)  
# With validate = True it doesn't find any subjects
participants = bidslayout.get_subjects()

fd_mean, name_of_files, fd_count_high = [], [], []
for i in  participants:
    print("Subject: ", i)
    i = i.replace("sub-", "")
    for ses in bidslayout.get_sessions(subject=i):
        print("Session: ", ses)
        #for multi-echo files, apply transform to standard space
        for r in bidslayout.get_runs(subject=i, session=ses):
            print("Run:", r)

            # setting up the data and cheking the existance afterwards for BOLD images
            filesrest = bidslayout.get(subject=i,
                            session=ses,
                            run=r,
                            extension=".nii.gz",
                            suffix="bold",
                            space="MNI152NLin2009cAsym",
                            #regex_search=True,
                            #invalid_filters="allow"
                            )

            print("Filerest:", filesrest)
            if len(filesrest)>1:
                print(f"More than one file match bold filters. Subject {i}, ses {ses}, run {r}")
                continue
            elif len(filesrest)<1:
                print(f"No file match bold filters. Subject {i}, ses {ses}, run {r}")
                continue
            bold = filesrest[0]
            
            #Load Confounds
            confounds_ents = {}
            confounds_ents["subject"] = bold.entities['subject']
            confounds_ents["session"] = bold.entities['session']
            #confounds_ents["acquisition"] = bold.entities['acquisition']
            confounds_ents["run"] = bold.entities['run']
            confounds_ents["task"] = bold.entities['TaskName']
            #confounds_ents['desc'] = "confounds"
            confounds_ents['suffix'] = "timeseries"
            confounds_ents['extension'] = ".tsv"
            confounds = bidslayout.get(return_type='file', **confounds_ents, invalid_filters="allow")[0]
            allconfounds_df = pd.read_csv(confounds, sep='\t')

            # this is from https://doi.org/10.1073/pnas.1502829112 article, from https://wiki.cam.ac.uk/bmuwiki/FMRI website
            fd_count_high.append(np.sum(allconfounds_df['framewise_displacement'] > 0.25) > 20)
            fd_mean.append(allconfounds_df['framewise_displacement'].mean()> 0.2)
            name_of_files.append(bold.filename)

sub_list_fd = pd.DataFrame({'file_name': name_of_files,'fd_mean': fd_mean  , 'fd_count_high': fd_count_high })


Rawdata:  c:\Users\angel\Documents\pilotStudy_INPD\pilotStudydata


c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\layout\layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([WindowsPath('c:/Users/angel/Documents/pilotStudy_INPD/pilotStudydata/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


Subject:  00003
Session:  1
Run: 1
Filerest: [<BIDSImageFile filename='c:\Users\angel\Documents\pilotStudy_INPD\pilotStudydata\sub-00003\ses-1\func\sub-00003_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'>]
Session:  3
Run: 1
Filerest: [<BIDSImageFile filename='c:\Users\angel\Documents\pilotStudy_INPD\pilotStudydata\sub-00003\ses-3\func\sub-00003_ses-3_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'>]
Subject:  00008
Session:  1
Run: 1
Filerest: [<BIDSImageFile filename='c:\Users\angel\Documents\pilotStudy_INPD\pilotStudydata\sub-00008\ses-1\func\sub-00008_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'>]
Subject:  00015
Session:  1
Run: 1
Filerest: [<BIDSImageFile filename='c:\Users\angel\Documents\pilotStudy_INPD\pilotStudydata\sub-00015\ses-1\func\sub-00015_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'>]
Subject:  00016
Session:  1
Run: 1
Filerest: [<BIDSImage

In [ ]:
sub_list_fd.to_csv(os.path.join(output,'sub_list_fd.csv'), index=False)

In [11]:
### this cell separates the subjects and session of the frame displacement data
abspath = os.path.abspath('__file__')
dname = os.path.dirname(os.path.dirname(abspath))
os.environ["ROOTDIR"] = dname  # seth path
rootdir = os.environ["ROOTDIR"]

datasource = os.path.join(rootdir, 'metadata')

data = pd.read_csv(os.path.join(datasource,'sub_list_fd.csv'))
sub_ses_df = data['file_name'].str.split('/', expand=True).drop([0,1,2,3,4,5,8,9],axis=1)
fd_df = data.drop(['file_name'],axis=1)

final_data = pd.concat([sub_ses_df,fd_df],axis=1)
final_data = final_data.rename(columns={6:'sub_id' , 7:'session'})
final_data

In [42]:
pd.read_csv()

,sub_id,session,fd_mean,fd_count_high
0,sub-00878,ses-2,True,True
1,sub-00878,ses-3,False,False
2,sub-00878,ses-1,False,False
3,sub-01659,ses-1,False,False
4,sub-01610,ses-2,True,True
...,...,...,...,...
1525,sub-01727,ses-1,False,False
1526,sub-00354,ses-1,False,False
1527,sub-00937,ses-2,False,True
1528,sub-00937,ses-3,False,False
